In [1]:
# Install required libraries
!pip install transformers pandas openpyxl


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from transformers import XLNetTokenizer, XLNetModel
import torch

# Load XLNet model and tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

In [3]:
# Load your long text dataset from Excel
excel_file_path = 'TEST FAKE N NOT FAKE list.xlsx'
df = pd.read_excel(excel_file_path)

# Tokenize and embed each text in the dataset
embedded_vectors = []

for text in df['cleaned_desc']:
    # Tokenize the text
    tokens = tokenizer.encode(text, return_tensors='pt', max_length=600, truncation=True)
    
    # Get the model's output for the tokenized text
    with torch.no_grad():
        output = model(tokens)

    # Extract the embeddings from the output
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()

    # Append the embedded vector to the list
    embedded_vectors.append(embeddings)

In [4]:
# Create a new column in the DataFrame for the embedded vectors
df['embedded_text_vectors'] = embedded_vectors

In [5]:
df.head()

,id,cluster,id.1,Price,Built_Up_SF,Bathroom,Furnishing,Bedroom,Tenure,Car_Park,...,label,label2,image_name,image_name2,Filename,label_preprocess,label_preprocess2,filename,Unnamed: 2,embedded_text_vectors
0,243,0,244,360720,872,2,0,2,1,3,...,0,notfake,304192811980656.jpg,304192811980656,304192811980656.jpg,FAKE,1,304192811980656.jpg,jpg,"[-0.7801213, 0.5439537, -1.3790228, 0.5155359,..."
1,867,0,876,630448,988,2,1,3,0,2,...,0,notfake,299833811679215.png,299833811679215,299833811679215.png,FAKE,1,299833811679215.png,png,"[-0.9799025, 0.3343956, -1.0918189, 0.9262733,..."
2,1475,0,1488,904800,1001,2,1,3,1,2,...,0,notfake,299931411684815.jpeg,299931411684815,299931411684815.jpeg,FAKE,1,299931411684815.jpeg,jpeg,"[-0.7511221, 0.012002154, -1.0715668, 1.211338..."
3,1743,0,1760,4323000,3606,3,0,3,0,3,...,0,notfake,301900811820080.jpg,301900811820080,301900811820080.jpg,FAKE,1,301900811820080.jpg,jpg,"[-0.97380567, 0.27740255, -0.88993174, 0.89790..."
4,1964,0,1982,2000000,2898,3,0,3,1,3,...,0,notfake,305103412044624.jpg,305103412044624,305103412044624.jpg,FAKE,1,305103412044624.jpg,jpg,"[-0.80897975, 0.4114051, -1.6031532, 1.1741426..."


In [6]:
# Assuming 'df' is your DataFrame and 'embedded_vectors' is the column of interest
num_vectors = df['embedded_text_vectors'].apply(len)
num_vectors_count = num_vectors.max()  # Assuming all vectors have the same length, otherwise use min()

# Print or use 'num_vectors_count' as needed
print(f"The number of vectors in the 'embedded_vectors' column is: {num_vectors_count}")

The number of vectors in the 'embedded_vectors' column is: 768


In [7]:
print(df['embedded_text_vectors'])

0      [-0.7801213, 0.5439537, -1.3790228, 0.5155359,...
1      [-0.9799025, 0.3343956, -1.0918189, 0.9262733,...
2      [-0.7511221, 0.012002154, -1.0715668, 1.211338...
3      [-0.97380567, 0.27740255, -0.88993174, 0.89790...
4      [-0.80897975, 0.4114051, -1.6031532, 1.1741426...
                             ...                        
665    [-0.8056097, 0.24576299, -1.4443183, 0.7272955...
666    [-0.53082126, -0.10427194, -0.90094084, 0.6077...
667    [-1.1265523, 0.2756029, -0.79708225, 0.8449295...
668    [-0.8921038, 0.7888857, -1.5612562, 0.75990623...
669    [-0.9186594, 0.26485527, -1.0769023, 0.9139204...
Name: embedded_text_vectors, Length: 670, dtype: object


In [8]:
import pandas as pd
import tensorflow as tf

# Assuming df is your DataFrame with the 'embedded_vector' column
# Replace the ellipsis (...) with the actual column name if it's different
#df = pd.DataFrame('embedded_array2')

# Convert the entire 'embedded_vector' column to a TensorFlow tensor
text_tensor_array = tf.convert_to_tensor(df['embedded_text_vectors'].tolist(), dtype=tf.float32)

# Use tensor_array in your TensorFlow model training
# For example, you can use it as input to the model.fit() method

In [9]:
print(text_tensor_array)

tf.Tensor(
[[-0.7801213   0.5439537  -1.3790228  ... -0.60438246  0.27426016
  -0.22424637]
 [-0.9799025   0.3343956  -1.0918189  ... -0.59129167  0.6083535
  -0.3939812 ]
 [-0.7511221   0.01200215 -1.0715668  ... -1.3098949  -0.34847164
  -0.23657413]
 ...
 [-1.1265523   0.2756029  -0.79708225 ... -0.83931595 -0.3924915
  -0.32212827]
 [-0.8921038   0.7888857  -1.5612562  ... -0.87483585 -0.30020982
  -0.24282107]
 [-0.9186594   0.26485527 -1.0769023  ... -1.1286374  -0.6740539
  -0.19209328]], shape=(670, 768), dtype=float32)


In [10]:
import os
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [11]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input

# Specify the depth of ResNet (n can be 50, 101, 152, etc.)
#n = 101

# Load ResNet model
resnet_model = ResNet101(weights='imagenet', include_top=False, pooling='avg' )

# You can then use this model for image embedding or fine-tune it for your specific task

In [12]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

# Function to extract ResNet embeddings
def extract_resnet_embeddings(image_path, model):
    img_array = load_and_preprocess_image(image_path)
    img_array = img_array.reshape((1,) + img_array.shape)
    embeddings = model.predict(img_array)
    return embeddings.flatten()

In [13]:
# Path to the folder containing images
image_folder = 'TEST FAKE N NOT FAKE'

In [14]:
# Create empty lists to store embeddings and labels
embeddings_list = []
labels_list = []

# Iterate through the DataFrame and extract embeddings
for index, row in df.iterrows():
    filename = row['Filename']
    label = row['label_preprocess2']
    image_path = os.path.join(image_folder, filename)
    
    # Check if the image file exists
    if os.path.exists(image_path):
        # Extract ResNet embeddings
        embeddings = extract_resnet_embeddings(image_path, resnet_model)
        
        # Append embeddings and labels to the lists
        embeddings_list.append(embeddings)
        labels_list.append(label)

1/1 [==============================] - 0s 82ms/step


1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 87ms/step


In [15]:
# Create a DataFrame for the embeddings
embeddings_df = pd.DataFrame(data={'Embedded_Image_Vector': embeddings_list})

# Add the 'label' column to the DataFrame
embeddings_df['label_preprocess2'] = labels_list

# Display the DataFrame with embeddings and labels
print(embeddings_df)

                                 Embedded_Image_Vector  label_preprocess2
0    [0.0105281705, 0.41969872, 0.2552484, 0.007083...                  1
1    [0.09064932, 0.07180481, 0.29487756, 0.0, 0.56...                  1
2    [0.010657048, 0.60193026, 0.0, 0.04426891, 0.0...                  1
3    [0.08432497, 0.5170417, 0.178128, 0.03865541, ...                  1
4    [0.33428207, 0.23564343, 0.0, 0.34269917, 0.16...                  1
..                                                 ...                ...
665  [0.29429737, 0.2465152, 0.1813353, 0.07788115,...                  0
666  [0.21608119, 0.28764632, 0.023309898, 0.0, 0.0...                  0
667  [0.4463434, 1.7051865, 0.001198501, 0.03668768...                  0
668  [0.22571178, 0.67161196, 0.18863422, 0.0, 0.51...                  0
669  [0.1663687, 1.0618463, 0.010455365, 0.22656396...                  0

[670 rows x 2 columns]


In [16]:
X=embeddings_df['Embedded_Image_Vector']

In [17]:
# Import numpy
import numpy as np
# Create an empty list to store tensors
batch_size = 1000
image_tensor_list = []

for i in range(0, len(X), batch_size):
    batch_vectors = X[i:i+batch_size]
    # Assuming batch_vectors is a list of arrays with varying shapes
    # Convert each array to a flat array with consistent shape
    batch_vectors_flat = [np.ravel(vec) for vec in batch_vectors]

    # Convert the list of flat arrays to a 2D NumPy array
    tensor_data = np.array(batch_vectors_flat, dtype=np.float32)

    # Convert the NumPy array to a TensorFlow tensor and append to the list
    image_tensor_list.append(tf.convert_to_tensor(tensor_data))

In [18]:
print(image_tensor_list)

[<tf.Tensor: shape=(670, 2048), dtype=float32, numpy=
array([[1.05281705e-02, 4.19698715e-01, 2.55248398e-01, ...,
        3.34575512e-02, 4.31762040e-02, 5.97864211e-01],
       [9.06493217e-02, 7.18048066e-02, 2.94877559e-01, ...,
        4.71669808e-03, 0.00000000e+00, 0.00000000e+00],
       [1.06570479e-02, 6.01930261e-01, 0.00000000e+00, ...,
        0.00000000e+00, 2.46839449e-01, 1.36168212e-01],
       ...,
       [4.46343392e-01, 1.70518649e+00, 1.19850098e-03, ...,
        1.47448695e+00, 3.71041238e-01, 1.29536903e-02],
       [2.25711778e-01, 6.71611965e-01, 1.88634217e-01, ...,
        3.15330505e-01, 1.49466336e-01, 4.94063824e-01],
       [1.66368693e-01, 1.06184626e+00, 1.04553653e-02, ...,
        9.48606074e-01, 1.97069556e-01, 6.54933229e-02]], dtype=float32)>]


In [19]:
# Assuming tensor_list is your list of TensorFlow tensors
merged_tensor_image = tf.concat(image_tensor_list, axis=0)

In [20]:
print(merged_tensor_image)

tf.Tensor(
[[1.05281705e-02 4.19698715e-01 2.55248398e-01 ... 3.34575512e-02
  4.31762040e-02 5.97864211e-01]
 [9.06493217e-02 7.18048066e-02 2.94877559e-01 ... 4.71669808e-03
  0.00000000e+00 0.00000000e+00]
 [1.06570479e-02 6.01930261e-01 0.00000000e+00 ... 0.00000000e+00
  2.46839449e-01 1.36168212e-01]
 ...
 [4.46343392e-01 1.70518649e+00 1.19850098e-03 ... 1.47448695e+00
  3.71041238e-01 1.29536903e-02]
 [2.25711778e-01 6.71611965e-01 1.88634217e-01 ... 3.15330505e-01
  1.49466336e-01 4.94063824e-01]
 [1.66368693e-01 1.06184626e+00 1.04553653e-02 ... 9.48606074e-01
  1.97069556e-01 6.54933229e-02]], shape=(670, 2048), dtype=float32)


In [21]:
print(df['label_preprocess2'])

0      1
1      1
2      1
3      1
4      1
      ..
665    0
666    0
667    0
668    0
669    0
Name: label_preprocess2, Length: 670, dtype: int64


In [22]:
y_test_featuresfusion = df['label_preprocess2'].values 

In [23]:
print(y_test_featuresfusion)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [24]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
from sklearn.preprocessing import StandardScaler  # Use appropriate scaler if needed

# Load the trained model
model = load_model('model_CONCATENATE3_dense200_resnet101.h5')  # Replace 'path_to_desktop' with the actual path

In [25]:
# Predict probabilities
y_pred_proba_cnn = model.predict([text_tensor_array, merged_tensor_image])

21/21 [==============================] - 0s 2ms/step


In [26]:
# Assuming you want the predicted class labels (0 or 1)
y_pred_labels_featuresfusion = np.argmax(y_pred_proba_cnn, axis=1)

In [27]:
# Print classification report
print("Classification Report:")
print(classification_report(y_test_featuresfusion, y_pred_labels_featuresfusion))

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       555
           1       0.86      0.49      0.62       115

    accuracy                           0.90       670
   macro avg       0.88      0.74      0.78       670
weighted avg       0.90      0.90      0.89       670



In [28]:
# Print confusion matrix
conf_matrix = confusion_matrix(y_test_featuresfusion, y_pred_labels_featuresfusion)  
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[546   9]
 [ 59  56]]


In [29]:
print(y_pred_proba_cnn)

[[0.18303794 0.81696206]
 [0.12171806 0.87828195]
 [0.09781878 0.9021812 ]
 ...
 [0.99607617 0.00392386]
 [0.9957071  0.00429296]
 [0.9983028  0.00169712]]


In [ ]:
##### xgboost optimum ###

In [30]:
pip install pandas scikit-learn xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [32]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [33]:
import pandas as pd

In [34]:
# Load the dataset from Excel file
file_path_train_xgboost = 'TRAIN FAKE N NOT FAKE list.xlsx'
df_train_xgboost = pd.read_excel(file_path_train_xgboost)

# Assume 'label' column is the target variable
target_column = 'label_preprocess2'

In [35]:
# Split the data into features (X) and target variable (y)
X_train_xgboost = df_train_xgboost[['cluster','Built_Up_SF','Bathroom','Furnishing','Bedroom','Tenure','Car_Park','Negeri','Property_Type','Latitude','Longitude','Occupancy','Unit_Type']]
y_train_xgboost = df_train_xgboost[target_column]

In [36]:
# Encode the target variable 'label'
label_encoder = LabelEncoder()
y_encoded_train_xgboost = label_encoder.fit_transform(y_train_xgboost)

# Apply feature normalization (MinMaxScaler)
scaler = MinMaxScaler()
X_scaled_train_xgboost = scaler.fit_transform(X_train_xgboost)

In [37]:
# Train an XGBoost model
params = {
    'objective': 'multi:softmax',
    'num_class': 2,
    'max_depth': 8,
    'n_estimators': 300,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'merror'
}

In [38]:
num_rounds = 100
dtrain = xgb.DMatrix(X_scaled_train_xgboost, label=y_encoded_train_xgboost)
model = xgb.train(params, dtrain, num_rounds)

/Users/maifuza/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [09:21:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [39]:
# Save the model
model.save_model("xgboost_model_pretrained.model")

/Users/maifuza/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [09:21:46] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [40]:
# Load the model for inference
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model_pretrained.model")

In [41]:
file_path_test_xgboost = 'TEST FAKE N NOT FAKE list.xlsx'
df_test_xgboost = pd.read_excel(file_path_test_xgboost)

# Assume 'label' column is the target variable
target_column_test = 'label_preprocess2'

In [42]:
X_test_xgboost = df_test_xgboost[['cluster','Built_Up_SF','Bathroom','Furnishing','Bedroom','Tenure','Car_Park','Negeri','Property_Type','Latitude','Longitude','Occupancy','Unit_Type']]
y_test_xgboost = df_test_xgboost[target_column_test]

In [43]:
# Encode the target variable 'label'
label_encoder = LabelEncoder()
y_encoded_test_xgboost = label_encoder.fit_transform(y_test_xgboost)

# Apply feature normalization (MinMaxScaler)
scaler = MinMaxScaler()
X_scaled_test_xgboost = scaler.fit_transform(X_test_xgboost)

In [44]:
# Create DMatrix for test data
dtest = xgb.DMatrix(X_scaled_test_xgboost)

In [45]:
# Predict using the loaded model
predictions = loaded_model.predict(dtest)

In [46]:
# Evaluate the predictions
accuracy = accuracy_score(y_encoded_test_xgboost, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8865671641791045


In [47]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [48]:
# Convert the class labels to strings
class_names = [str(label) for label in label_encoder.classes_]

# Evaluate the performance
classification_rep = classification_report(y_encoded_test_xgboost, predictions, target_names=class_names)
confusion_matrix = confusion_matrix(y_encoded_test_xgboost, predictions)

In [49]:
# Print the results
print(f'Accuracy: {accuracy:.4f}')
print('\nClassification Report:')
print(classification_rep)

Accuracy: 0.8866

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       555
           1       0.69      0.63      0.65       115

    accuracy                           0.89       670
   macro avg       0.80      0.78      0.79       670
weighted avg       0.88      0.89      0.88       670



In [50]:
# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix)


Confusion Matrix:
[[522  33]
 [ 43  72]]


In [51]:
print(predictions)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [52]:
# Round the predicted probabilities to the nearest integer
binary_predictions = [int(round(pred)) for pred in predictions]

# Print the binary predictions
print("Binary Predictions:", binary_predictions)

Binary Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [53]:
encoded_binary_predictions_xgboost = label_encoder.fit_transform(binary_predictions)

In [54]:
print(encoded_binary_predictions_xgboost)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [55]:
# Predict using the loaded model
predictions = loaded_model.predict(dtest, output_margin=True)

# Convert the raw output to probabilities using a softmax function
import numpy as np

def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

probabilities_xgb = softmax(predictions)

# Print the probabilities for the first few data points
for i in range(len(probabilities_xgb)):
    print(f"Data point {i+1} probabilities:", probabilities_xgb[i])


Data point 1 probabilities: [0.8522485  0.14775154]
Data point 2 probabilities: [0.7207512 0.2792488]
Data point 3 probabilities: [0.9201199  0.07988012]
Data point 4 probabilities: [0.9156811  0.08431891]
Data point 5 probabilities: [0.96470463 0.03529539]
Data point 6 probabilities: [0.6458633  0.35413674]
Data point 7 probabilities: [0.67316234 0.32683772]
Data point 8 probabilities: [0.9760495  0.02395051]
Data point 9 probabilities: [0.8944427  0.10555731]
Data point 10 probabilities: [0.6302541 0.3697459]
Data point 11 probabilities: [0.975109   0.02489101]
Data point 12 probabilities: [0.7110162  0.28898382]
Data point 13 probabilities: [0.84986603 0.15013395]
Data point 14 probabilities: [0.6762641 0.3237359]
Data point 15 probabilities: [0.9157448  0.08425517]
Data point 16 probabilities: [0.7583912  0.24160878]
Data point 17 probabilities: [0.9499305  0.05006947]
Data point 18 probabilities: [0.7954097 0.2045903]
Data point 19 probabilities: [0.64718026 0.35281974]
Data point

In [56]:
print(probabilities_xgb)

[[0.8522485  0.14775154]
 [0.7207512  0.2792488 ]
 [0.9201199  0.07988012]
 ...
 [0.5722724  0.42772764]
 [0.20668687 0.7933131 ]
 [0.3210523  0.6789477 ]]


In [ ]:
##### end of xgboost optimum ####

In [57]:
print(y_pred_labels_featuresfusion)

[1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1
 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1
 1 0 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [58]:
print(y_encoded_test_xgboost)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [59]:
import numpy as np

In [60]:
def combine_beliefs(m_a, m_b, weights):
    """
    Combines beliefs from two sources using Dempster's rule of combination with class weights.
    """
    # Calculate the conflict measure
    K = weights[0] * m_a[1] * m_b[0] + weights[1] * m_a[0] * m_b[1]

    # Calculate combined belief
    m_combined = [(weights[0] * m_a[0] * m_b[0]) / (1 - K), (weights[1] * m_a[1] * m_b[1]) / (1 - K)]

    return m_combined

def predict_labels(m_combined):
    """
    Predicts the labels based on combined beliefs.
    """
    if m_combined[0] > m_combined[1]:
        return "Not Fake"
    else:
        return "Fake"

In [61]:
print(y_pred_proba_cnn)

[[0.18303794 0.81696206]
 [0.12171806 0.87828195]
 [0.09781878 0.9021812 ]
 ...
 [0.99607617 0.00392386]
 [0.9957071  0.00429296]
 [0.9983028  0.00169712]]


In [77]:
# Example probabilities from two sources (you should replace these with your actual data)
y_pred_proba_cnn = np.array(y_pred_proba_cnn)  # Replace with your data
y_pred_proba_xgboost = np.array(probabilities_xgb) # Replace with your data

# Define class weights for each class
weights = [0.1, 1.0]  # Example: weight for 'not fake' class is 0.3, weight for 'fake' class is 0.7

In [78]:
# Initialize empty lists to store combined beliefs and predictions
combined_beliefs = []
predictions = []

# Combine beliefs and make predictions for each data point
for prob_cnn, prob_xgboost in zip(y_pred_proba_cnn, y_pred_proba_xgboost):
    m_a = prob_cnn
    m_b = prob_xgboost

    # Combine beliefs with class weights
    m_combined = combine_beliefs(m_a, m_b, weights)
    combined_beliefs.append(m_combined)

    # Predict the label
    prediction = predict_labels(m_combined)
    predictions.append(prediction)

# Print combined beliefs and predictions
for i, (belief, prediction) in enumerate(zip(combined_beliefs, predictions)):
    print(f"Data point {i+1}: Combined Beliefs - {belief}, Prediction - {prediction}")

Data point 1: Combined Beliefs - [0.017268743095095632, 0.13362486601219858], Prediction - Fake
Data point 2: Combined Beliefs - [0.009718362348707196, 0.2716926823621532], Prediction - Fake
Data point 3: Combined Beliefs - [0.009899638061568804, 0.07926566669977081], Prediction - Fake
Data point 4: Combined Beliefs - [0.02106674529926049, 0.0732643403385392], Prediction - Fake
Data point 5: Combined Beliefs - [0.046729431390501765, 0.0208095620753662], Prediction - Not Fake
Data point 6: Combined Beliefs - [0.03952528493765062, 0.22895022968500237], Prediction - Fake
Data point 7: Combined Beliefs - [0.006028949180306459, 0.3293000845372065], Prediction - Fake
Data point 8: Combined Beliefs - [0.08482695568805489, 0.004027083105003364], Prediction - Not Fake
Data point 9: Combined Beliefs - [0.09955975602446886, 0.0005103535538772848], Prediction - Not Fake
Data point 10: Combined Beliefs - [0.05272530448477445, 0.18655390409281272], Prediction - Fake
Data point 11: Combined Beliefs -

In [79]:
# Convert string labels to numerical values
numerical_predictions = [1 if label == 'Fake' else 0 for label in predictions]

# Print numerical predictions
print(numerical_predictions)

[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [80]:
print(y_test_featuresfusion)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [81]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_featuresfusion, numerical_predictions))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_featuresfusion, numerical_predictions))


Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       555
           1       0.82      0.74      0.78       115

    accuracy                           0.93       670
   macro avg       0.88      0.85      0.87       670
weighted avg       0.92      0.93      0.93       670


Confusion Matrix:
[[536  19]
 [ 30  85]]


In [ ]:
import numpy as np

def combine_beliefs(m_a, m_b, weights):
    """
    Combines beliefs from two sources using Dempster's rule of combination with class weights.
    """
    # Calculate the conflict measure
    K = weights[0] * m_a[1] * m_b[0] + weights[1] * m_a[0] * m_b[1]

    # Calculate combined belief
    m_combined = [(weights[0] * m_a[0] * m_b[0]) / (1 - K), (weights[1] * m_a[1] * m_b[1]) / (1 - K)]

    return m_combined

def predict_labels(m_combined):
    """
    Predicts the labels based on combined beliefs.
    """
    if m_combined[0] > m_combined[1]:
        return "Not Fake"
    else:
        return "Fake"

# Example probabilities from two sources (you should replace these with your actual data)
y_pred_proba_cnn = np.array(y_pred_proba_cnn)  # Replace with your data
y_pred_proba_xgboost = np.array(y_pred_proba_xgboost) # Replace with your data

# Define class weights for each class
weights = [0.3, 0.7]  # Example: weight for 'not fake' class is 0.3, weight for 'fake' class is 0.7

# Initialize empty lists to store combined beliefs and predictions
combined_beliefs = []
predictions = []

# Combine beliefs and make predictions for each data point
for prob_cnn, prob_xgboost in zip(y_pred_proba_cnn, y_pred_proba_xgboost):
    m_a = prob_cnn
    m_b = prob_xgboost

    # Combine beliefs with class weights
    m_combined = combine_beliefs(m_a, m_b, weights)
    combined_beliefs.append(m_combined)

    # Predict the label
    prediction = predict_labels(m_combined)
    predictions.append(prediction)

# Print combined beliefs and predictions
for i, (belief, prediction) in enumerate(zip(combined_beliefs, predictions)):
    print(f"Data point {i+1}: Combined Beliefs - {belief}, Prediction - {prediction}")
